In [67]:
import tqdm
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [68]:
# Read the db file and clustering results
db_file = pd.read_csv("../susdat_2022-08-19-155358.csv",sep="\t")
feature_file_name = "analysis_results/coverage0.9_correlation0.6.csv"
clustered_data_file = pd.read_csv(feature_file_name)

/tmp/ipykernel_106986/1683423195.py:2: DtypeWarning: Columns (3,5,6,9,10,11,12,13,15,18,19,20,21,22,23,30,32,33,35,38,41,44,46,47,48,49,50,51,57,61,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  db_file = pd.read_csv("../susdat_2022-08-19-155358.csv",sep="\t")


In [69]:
db_file.columns

Index(['Norman_SusDat_ID', 'Name', 'Name_Dashboard', 'Name_ChemSpider',
       'Name_IUPAC', 'Synonyms_ChemSpider',
       'Reliability_of_Synonyms_ChemSpider', 'CAS_RN', 'CAS_RN_Dashboard',
       'CAS_RN_PubChem', 'CAS_RN_Cactus', 'CAS_RN_ChemSpider',
       'Reliability_of_CAS_ChemSpider', 'Validation_Level', 'SMILES',
       'SMILES_Dashboard', 'StdInChI', 'StdInChIKey', 'MS_Ready_SMILES',
       'MS_Ready_StdInChI', 'MS_Ready_StdInChIKey', 'Source', 'PubChem_CID',
       'ChemSpiderID', 'DTXSID', 'Molecular_Formula', 'Monoiso_Mass', 'M+H+',
       'M-H-', 'Pred_RTI_Positive_ESI', 'Uncertainty_RTI_pos',
       'Pred_RTI_Negative_ESI', 'Uncertainty_RTI_neg',
       'Tetrahymena_pyriformis_toxicity', 'IGC50_48_hr_ug/L',
       'Uncertainty_Tetrahymena_pyriformis_toxicity', 'Daphnia_toxicity',
       'LC50_48_hr_ug/L', 'Uncertainty_Daphnia_toxicity', 'Algae_toxicity',
       'EC50_72_hr_ug/L', 'Uncertainty_Algae_toxicity',
       'Pimephales_promelas_toxicity', 'LC50_96_hr_ug/L',
    

In [70]:
def calculate_ppm_error_between_two_masses(exact_mass,observed_mass): # Calculate ppm error between a two masses
    mass_difference = observed_mass-exact_mass
    ppm_error = (mass_difference * 1000000) / exact_mass
    return (ppm_error)

all_masses = db_file["M-H-"].to_list()
log_k = db_file["logKow_EPISuite"].to_list()

In [73]:
ppm_error_cutoff = 5
count = 0
output_results = []
# Compare the M-H- value to the m/z and get the hits and consoliate the results
for index,row in tqdm.tqdm(clustered_data_file.iterrows(),leave=True,position=0,total=len(clustered_data_file)):
    if row["Met_ID"]:
        mz_cf = row["mz_cf"]
        hits = [i for i,mass in enumerate(all_masses) if abs(calculate_ppm_error_between_two_masses(mz_cf,mass)) <= ppm_error_cutoff]
        names = ""
        for indexes in hits:
            names += db_file.iloc[indexes]["Name"] + " % "
        if len(hits) > 0:
            count += 1
            met_id_found_names = {}
            met_id_found_names["met_id"] = int(row["Met_ID"])
            met_id_found_names["mz_cf"] = mz_cf
            met_id_found_names["rt_cf"] = row["rt_cf"]
            met_id_found_names["label"] = row["label"]
            met_id_found_names["intensity_cf"] = row["intensity_cf"]
            
            log_k_list = ""
            for indexes in hits:
                log_k_list += str(log_k[indexes]) + " % "
            met_id_found_names["log_k"] = log_k_list
            
            if row["real_name"] == "nan":
                met_id_found_names["real_name"] = "NA"
            else:
                met_id_found_names["real_name"] = row["real_name"]
            met_id_found_names["names"] = str(names)
            output_results.append(met_id_found_names)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 81.83it/s]


In [74]:
# Read native compound files and fetch necessary values to make a lm model
native_cmpds = pd.read_csv("all_no_Br_Cl.csv")
native_cmpds = native_cmpds[native_cmpds["Native/IS"] == "Native"]
names = native_cmpds["Abbreviation"].to_list()
rt_values =  [(entry) for entry in native_cmpds["Rt(s)"].to_list()]
mz_values =  [(entry) for entry in native_cmpds["MIM"].to_list()]
log_k_list = [(entry) for entry in native_cmpds["Unnamed: 11"].to_list()]

In [75]:
# make rt mz feature
rt_mz_values = [[rt,mz] for rt,mz in zip(rt_values,mz_values)]

In [76]:
rt_list = np.array(rt_values)
rt_list = rt_list.reshape(-1,1)
rt_list = rt_mz_values
# Create linear regression object
regr = linear_model.LinearRegression()
regr.fit(rt_list, log_k_list)
y_pred = regr.predict(rt_list)

# The coefficients
print("Coefficients: ", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(log_k_list, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(log_k_list, y_pred))

Coefficients:  [0.00223999 0.0111889 ]
Mean squared error: 0.87
Coefficient of determination: 0.77


In [63]:
# Making a df from the output_results
# File containing met_id	mz_cf	rt_cf	label	intensity_cf	log_k	real_name	names
out_df = pd.DataFrame.from_dict(output_results)
out_df

,met_id,mz_cf,rt_cf,label,intensity_cf,log_k,real_name,names
0,102,462.963334,470.159358,816,261810.40,5.48 % 5.67 % 5.86 % 6.64 %,PFNA,Perfluorononanoic acid (PFNA) % Perfluoro-6-me...
1,173,512.960155,494.488819,816,113655.00,6.15 % 6.52 % 1.16 %,PFDA,"Perfluorodecanoic acid (PFDA) % Perfluoro-3,7-..."
2,527,498.930985,470.547127,816,2691497.00,4.49 % 4.68 % -4.19 %,PFOS,Perfluorooctanesulfonic acid (PFOS) % Isooctan...
3,1025,333.171717,459.013231,184,157106.00,4.83 %,NaN,Diisobutyl (4-methoxybenzylidene)malonate %
4,1069,398.936966,409.846603,184,1753351.00,3.16 %,PFHxS,Perfluorohexanesulfonic acid (PFHxS) %
5,1673,498.930997,461.647058,818,773771.20,4.49 % 4.68 % -4.19 %,NaN,Perfluorooctanesulfonic acid (PFOS) % Isooctan...
6,1772,368.976961,441.325040,817,155059.80,4.89 %,NaN,1-Hydroperfluoroheptane %
7,1981,412.966987,441.239461,817,482334.60,4.81 % 5.0 %,PFOA,Perfluorooctanoic acid (PFOA) % Perfluoro-6-(t...
8,3143,445.296514,544.073479,184,152372.10,5.88 %,NaN,Carpipramine %
9,3207,448.934703,443.220796,818,91080.49,3.82 % -1.86 %,L-PFHpS,Perfluoroheptanesulfonic acid (PFHpS) % Sodium...


In [64]:
# Predicting for each rt,mz values
for index,rows in out_df.iterrows():
    mz = rows["mz_cf"]
    rt = rows["rt_cf"]
    pred_log_k = regr.predict([[rt,mz]])[0]
    condition = (out_df["mz_cf"] == mz) & (out_df["rt_cf"] == rt)
    out_df.loc[condition, 'pred_log_k_value'] = round(pred_log_k,2)

In [65]:
# Writing output to a file
out_df = out_df[['met_id', 'mz_cf', 'rt_cf', 'label', 'intensity_cf', 'log_k','pred_log_k_value', 'real_name', 'names']]
out_df.to_csv(feature_file_name[:-4] + "_identification.tsv",sep="\t")